<a href="https://colab.research.google.com/github/ZacharyZekaiXu/ZekaiXu_CrossmodalRecognition/blob/main/SpikingMBmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
from google.colab import output, drive
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/MyDrive/BeeBRain2_TouchVision/

Mounted at /content/gdrive
/content/gdrive/MyDrive/BeeBRain2_TouchVision


In [ ]:
!pip install torch
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=274491 sha256=d5d5a4fd952fe9fc157f92df98be2202a06c6cf554493c287981f9fb422f60be
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
     |████████████████████████████████| 2.1 MB 5.0 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.15-cp37-cp37m-linux_x86_64.whl size=516860 sha256=67a694bb00443f8802c2029a2cae49d3b91276fff4b306efb5e40ae649636fb3
  Stored in directory: /root/.cache/pip/whe

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import warnings
import os
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sn
import scipy.spatial as ss

from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, ChebConv, TAGConv
from torch_geometric.utils import normalized_cut
from torch_geometric.nn import (SplineConv, graclus, max_pool, max_pool_x, global_mean_pool)

In [ ]:
from __future__ import print_function

import time
from datetime import date
import torchvision.transforms as transforms
import numpy as np
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import random
import tqdm

In [ ]:
!pip install brian2
from brian2 import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Construct Graphs for Tactile Data [class: TactileGraph]

In [ ]:
class TactileGraph(object):

    def __init__(self, k=0, useKNN=0, dist_threshold=0): # we won't use kNN or MST there
        # 39 taxles in each tactile finger, with coordinates as:
        tact_coordinates = np.array([[-6, 0], [-5.3, -3], [-5.3, 3], [-4.6, -7.8], [-4.6, 7.8],
                    [-3.5, 0], [-3.05, -5.2], [-3.05, 5.2], [-3.1, -1.75], [-3.1, 1.75], # 10
                    [-1.6, -8.9], [-1.75, -3], [-1.6, 8.9], [-1.75, 3], [-1.5, 0],
                    [-0.7, -1.3], [-0.7, 1.3], [0, -6], [0, 6],[0, -3.5],
                    [0, 0], [0, 3.5], [0.8, -1.3], [0.8, 1.3], [1.6, -8.9],
                    [1.6, 8.9], [1.5, 0], [1.75, 3], [1.75, -3], [3.05, -5.2],
                    [3.1, 1.75], [3.05, 5.2], [3.1, -1.75], [3.6, 0], [4.6, -7.8], 
                    [4.6, 7.8], [5.3, -3], [5.3, 3], [6, 0]
                    ])
        self.pos = tact_coordinates

        if k == 0: # use manual way to construct the graph
            self.edge_origins = np.array([1, 1, 1, 2, 3, 6, 2, 2, 7, 9, 3, 3, 8, 10, 4, 4, 7, 11,
                    5, 5, 8, 13, 6, 6, 6, 9, 10, 15, 7, 7, 12, 18, 8, 8, 14, 19, 9, 9, 12, 16, 
                    10, 10, 14, 15, 11, 11, 18, 25, 12, 12, 16, 20, 13, 13, 19, 26, 
                    14, 14, 17, 22, 15, 15, 15, 16, 17, 21, 16, 16, 21, 23, 17, 17, 21, 24, 
                    18, 18, 18, 20, 25, 30, 19, 19, 19, 22, 26, 32, 20, 20, 23, 29, 
                    21, 21, 21, 23, 24, 27, 22, 28, 23, 23, 27, 29, 24, 24, 24, 27, 28, 31, 
                    25, 35, 26, 36, 27, 27, 33, 34, 28, 28, 31, 32, 29, 29, 30, 33, 30, 30, 35, 37,
                    31, 31, 34, 38, 32, 32, 36, 38, 33, 33, 34, 37, 34, 39, 37, 39, 38, 39]) - 1 
                    # minus 1 since taxel number from 0 
            self.edge_ends = np.array([2, 3, 6, 1, 1, 1, 7, 9, 2, 2, 8, 10, 3, 3, 7, 11, 4, 4, 
                    8, 13, 5, 5, 9, 10, 15, 6, 6, 6, 12, 18, 7, 7, 14, 19, 8, 8, 12, 16, 9, 9, 
                    14, 15, 10, 10, 18, 25, 11, 11, 16, 20, 12, 12, 19, 26, 13, 13, 
                    17, 22, 14, 14, 16, 17, 21, 15, 15, 15, 21, 23, 16, 16, 21, 24, 17, 17, 
                    20, 25, 30, 18, 18, 18, 22, 26, 32, 19, 19, 19, 23, 29, 20, 20,  
                    23, 24, 27, 21, 21, 21, 28, 22, 27, 29, 23, 23, 27, 28, 31, 24, 24, 24,
                    35, 25, 36, 26, 33, 34, 27, 27, 31, 32, 28, 28, 30, 33, 29, 29, 35, 37, 30, 30, 
                    34, 38, 31, 31, 36, 38, 32, 32, 34, 37, 33, 33, 39, 34, 39, 37, 39, 38]) - 1


    def getEdge(self):
        edges = torch.tensor([self.edge_origins, self.edge_ends])
        return edges #self.edge_origins, self.edge_ends

    def __call__(self, sample):
        graph_x = sample
        graph_edge_index = torch.tensor([self.edge_origins, self.edge_ends], dtype=torch.long)
        graph_pos = self.pos
        data = Data(x=graph_x, edge_index = graph_edge_index, pos=graph_pos)
#       data = []
        return data

    def __repr__(self):
        return "{}".format(self.__class__.__name__)

## Graph Convolution for Tactile Input

In [ ]:
thresh = 0.5    # neuronal threshold
lens = 0.5      # hyper-parameters of approximate function
decay = 0.2 

# define approximate firing function
class ActFun(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.gt(thresh).float()

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        temp = abs(input - thresh) < lens
        return grad_input * temp.float()

act_fun = ActFun.apply


# membrane potential update, for GCN
def mem_update_conv(ops, x, edge_idxs, mem, spike):
    mem = mem * decay * (1. - spike) + ops(x, edge_idxs)
    spike = act_fun(mem) # act_fun : approximation firing function
    return mem, spike

def mem_update(ops, x, mem, spike):
    mem = mem * decay * (1. - spike) + ops(x)
    spike = act_fun(mem)
    return mem, spike

# cnn_layer(in_channels, out_channels)
cfg_cnn = [(2, 64), #(64, 128)
] 

# kernel size
cfg_s = [39, 39]

# fc layer
cfg_fc = [128, 256]
gamma = 0.2 # dropout coefficient

print(cfg_cnn)
print(cfg_fc)

Model: TactileSGNet
[(2, 64)]
[128, 256]


### TactileSGNet

In [ ]:
class TactileSGNet(nn.Module):
    def __init__(self, num_classes=36, k=0, device="cuda:0"):
        super(TactileSGNet, self).__init__()
        in_planes, out_planes = cfg_cnn[0]
        self.conv1 = TAGConv(in_planes, out_planes, K=3)

        self.fc1 = nn.Linear(cfg_s[-1] * cfg_cnn[-1][1], cfg_fc[0])
        self.fc2 = nn.Linear(cfg_fc[0], cfg_fc[1])
        self.fc3 = nn.Linear(cfg_fc[1], num_classes)
        self.num_classes = num_classes
        self.graph = TactileGraph(k)
        self.device = device

    def forward(self, input, training = True):
        data = input.to(self.device)
        sizes = data.size()
        time_window = sizes[-1]

        c1_mem = c1_spike = c1_mem_noise = torch.zeros(cfg_s[0], cfg_cnn[0][1], device=self.device)
        h1_mem = h1_spike = h1_sumspike = h1_mem_noise = torch.zeros(cfg_fc[0], device=self.device)
        h2_mem = h2_spike = h2_sumspike = h2_mem_noise = torch.zeros(cfg_fc[1], device=self.device)
        h3_mem = h3_spike = h3_sumspike = h3_mem_noise = torch.zeros(self.num_classes, device=self.device)
        
        inputs = data.split(1, dim=len(sizes)-1)
        for step in range(time_window): # simulation time steps
            x = inputs[step].squeeze()
            x = x.to(self.device)
            graph_data = self.graph(x)
            x = graph_data.x.to(self.device) 
            edge_idxs = graph_data.edge_index.to(self.device)

            c1_mem, c1_spike = mem_update_conv(self.conv1, x, edge_idxs, c1_mem, c1_spike)
            x = c1_spike
            x = x.view(-1)

            h1_mem, h1_spike = mem_update(self.fc1, x, h1_mem, h1_spike)
            h1_sumspike += h1_spike
            h2_mem, h2_spike = mem_update(self.fc2, h1_spike, h2_mem,h2_spike)
            h2_sumspike += h2_spike
            h3_mem, h3_spike = mem_update(self.fc3, h2_spike, h3_mem, h3_spike)
            h3_sumspike += h3_spike

        outputs = h3_sumspike / time_window
        return outputs

## Training SGCN

In [ ]:

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
random.seed(0)

# model name
model_name = '_TactileSGNet_' # tactile 


# hyperparameter setting
num_classes = 36
k = 3 # number of nodes to be connected for constructing graph
num_run = 1
learning_rate = 1e-3 #1e-3
num_epochs = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Event-based tactile dataset
class tactileDataset(Dataset):
    def __init__(self, data_path, train=True):
        if train:
            self.files = os.listdir(data_path + '/train')
            self.file_path = data_path + '/train/'
        else:
            self.files = os.listdir(data_path + '/test')
            self.file_path = data_path + '/test/'

    def __getitem__(self, index):
        fileName = self.files[index]
        nameStr = fileName.split('_label_')
        label = int(nameStr[-1].split('.')[0])
        data = torch.from_numpy(np.load(self.file_path + fileName)) #torch.FloatTensor(np.load(self.file_path + fileName))
        label = torch.LongTensor([label])                      
        return data, label                                                                                                                 
    def __len__(self):
        return len(self.files)

# Decay learning rate
def lr_scheduler(optimizer, epoch, init_lr = 0.01, lr_decay_epoch=30):
    if epoch % lr_decay_epoch == 0 and epoch > 1:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
    return optimizer


# Tactile dataset
data_path = 'Ev-Objects/'
trainDataset = tactileDataset(data_path, train=True)
testDataset = tactileDataset(data_path, train=False)

# run for num_run times
best_acc = torch.zeros(num_run)
acc_list = list([])
training_loss_list = list([])
test_loss_list = list([])
net_list = list([])

for run in range(num_run):
    model = TactileSGNet(num_classes, k, device=device)
    model.to(device)
    criterion = nn.MSELoss() #nn.MSELoss(reduction='sum') #nn.BCELoss() 
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, amsgrad=True)

    acc = torch.zeros(num_epochs)
    training_loss = torch.zeros(num_epochs)
    test_loss = torch.zeros(num_epochs)
    for epoch in range(num_epochs):
        start_time = time.time()
        running_loss = 0
        for trainData, trainLabel in tqdm.tqdm(trainDataset): 
            model.zero_grad()
            optimizer.zero_grad()
            trainData = trainData.to(device) 
            outputs = model(trainData)
            labels_ = torch.zeros(1, num_classes).scatter_(1, trainLabel.view(-1, 1), 1)
            loss = criterion(outputs.cpu(), labels_) 
              
            running_loss += loss.item()
            loss.backward()
            optimizer.step()
            
        training_loss[epoch] = running_loss

         # testing
        correct = 0
        total = 0
        optimizer = lr_scheduler(optimizer, epoch, learning_rate, 40)
        running_loss = 0
        with torch.no_grad():
            for testData, testLabel in testDataset:
                optimizer.zero_grad()
                outputs = model(testData, False)
                labels_ = torch.zeros(1, num_classes).scatter_(1, testLabel.view(-1, 1), 1)
                loss = criterion(outputs.cpu(), labels_) 
                running_loss += loss.item()
                _, predicted = outputs.cpu().max(0)
                total += float(testLabel.size(0))
                correct += float(predicted.eq(testLabel).sum().item())
                
            test_loss[epoch] = running_loss         

            acc[epoch] = 100. * float(correct) / float(total)
            if best_acc[run] < acc[epoch]:
                best_acc[run] = acc[epoch]

        test_loss_list.append(test_loss) 
        training_loss_list.append(training_loss)

        acc_list.append(acc)
        if (epoch + 1) % 2 == 0:
            print('At epoch: %s, training_loss: %f, test_loss: %f, best accuracy: %.3f, time elasped: %.3f' % (epoch + 1, training_loss[epoch], test_loss[epoch], best_acc[run], time.time()-start_time ))
            start_time = time.time()
             
    net_list.append(model.state_dict())
    
# overall state
state = {
    'net_list': net_list,
    'best_acc': best_acc,
    'num_epochs': num_epochs,
    'acc_list': acc_list,
    'training_loss_list': training_loss_list,
    'test_loss_list': test_loss_list,
}
dateStr = date.today().strftime("%Y%m%d")

if not os.path.isdir('log_data'):
   os.mkdir('log_data')
torch.save(state, './log_data/' + dateStr + model_name + '_objects_' +  str(num_classes) + '_k_' + str(k)  + '.t7')
print('Avg acc: %f, std: %f: ' % (torch.mean(state['best_acc']), torch.std(state['best_acc'])))

In [ ]:
def model_PN(CPN, gLPN, ELPN, tau_IaPN, Ee, tau_syn_e, Ei, tau_syn_i, EIa, VtPN, VrPN, tau_ref, bPN):

    neuron_eqs = '''
    dv/dt = (g_l*(E_l-v) + g_e*(E_e-v) - g_i*(E_i-v) - g_Ia*(E_Ia-v) + I0)/C_m    : volt (unless refractory) # Ia is the spike triggered adaptation
    dg_e/dt = -g_e/tau_e  : siemens  # post-synaptic exc. conductance # synapses
    dg_i/dt = -g_i/tau_i  : siemens  # post-synaptic inh. conductance
    dg_Ia/dt = -g_Ia/tau_Ia : siemens # conductance adaptation 'current'
    I0 : amp
    '''

    neuron_modelPN = dict()
    neuron_modelPN['model'] = Equations(neuron_eqs, g_l=gLPN, E_l=ELPN, E_e=Ee, E_i=Ei,E_Ia = EIa, C_m=CPN, tau_e=tau_syn_e, tau_i=tau_syn_i,tau_Ia=tau_IaPN)
    neuron_modelPN['threshold'] = 'v > VtPN'
    neuron_modelPN['reset'] = '''v = VrPN; g_Ia-=bPN'''
    neuron_modelPN['refractory'] = tau_ref

    return neuron_modelPN

def model_LN(CLN, gLLN, ELLN, tau_IaLN, Ee, tau_syn_e, Ei, tau_syn_i, EIa, VtLN, VrLN, tau_ref, bLN):

    neuron_eqs = '''
    dv/dt = (g_l*(E_l-v) + g_e*(E_e-v) - g_i*(E_i-v) - g_Ia*(E_Ia-v) + I0)/C_m    : volt (unless refractory) # Ia is the spike triggered adaptation
    dg_e/dt = -g_e/tau_e  : siemens  # post-synaptic exc. conductance # synapses
    dg_i/dt = -g_i/tau_i  : siemens  # post-synaptic inh. conductance
    dg_Ia/dt = -g_Ia/tau_Ia : siemens # conductance adaptation 'current'
    I0 : amp
    '''

    neuron_modelLN = dict()
    neuron_modelLN['model'] = Equations(neuron_eqs, g_l=gLLN, E_l=ELLN, E_e=Ee, E_i=Ei, E_Ia=EIa, C_m=CLN, tau_e=tau_syn_e, tau_i=tau_syn_i, tau_Ia=tau_IaLN)
    neuron_modelLN['threshold'] = 'v > VtLN'
    neuron_modelLN['reset'] = '''v = VrLN; g_Ia-=bLN'''
    neuron_modelLN['refractory'] = tau_ref

    return neuron_modelLN


def model_KC(CKC, gLKC, ELKC, tau_IaKC, Ee, tau_syn_e, Ei, tau_syn_i, EIa, VtKC, VrKC, tau_ref, bKC):

    neuron_eqs = '''
    dv/dt = (g_l*(E_l-v) + g_e*(E_e-v) - g_i*(E_i-v) - g_Ia*(E_Ia-v) + I0)/C_m    : volt (unless refractory) # Ia is the spike triggered adaptation
    dg_e/dt = -g_e/tau_e  : siemens  # post-synaptic exc. conductance # synapses
    dg_i/dt = -g_i/tau_i  : siemens  # post-synaptic inh. conductance
    dg_Ia/dt = -g_Ia/tau_Ia : siemens # conductance adaptation 'current'
    I0 : amp
    '''

    neuron_modelKC = dict()
    neuron_modelKC['model'] = Equations(neuron_eqs, DeltaT=1 * mV, g_l=gLKC, E_l=ELKC, E_e=Ee, E_i=Ei, E_Ia=EIa, C_m=CKC, tau_e=tau_syn_e,tau_i=tau_syn_i, tau_Ia=tau_IaKC)
    neuron_modelKC['threshold'] = 'v > VtKC'
    neuron_modelKC['reset'] = '''v = VrKC; g_Ia-=bKC'''
    neuron_modelKC['refractory'] = tau_ref

    return neuron_modelKC



def network(params,
        input_ng,
        neuron_modelORN,
        neuron_modelPN,
        neuron_modelLN,
        neuron_modelKC,
        wORNinputORN,
        wORNPN,
        wORNLN,
        wLNPN,
        wPNKC,
        N_glu,
        ORNperGlu,
        N_KC,
        PNperKC,
        V0min,
        V0max,
        I0_PN = 0*nA,
        I0_LN = 0*nA,
        I0_KC = 0*nA,
        inh_delay=0 * ms):
    '''
    ## ToDo documentation ##
    Connect ORNs to PNs such that ORNperGlu ORNs representing input to one Glu connects to 1 PN
    repeat for every Glu, using connect_full. Connects ORNs to LNs in the same way.
    '''

    #########################     NEURONGROUPS     #########################

    NG = dict()

    # ORN Input
    #n_receptors = ORNperGlu * N_glu

    if input_ng is not None:
        validInputTypes = (PoissonGroup, Group, SpikeSource)
        assert isinstance(input_ng, validInputTypes), "parameter 'input_ng' must be of type: {}".format(validInputTypes)
        NG['ORNinput'] = input_ng

    neuron_params_orn = get_args_from_dict(neuron_modelORN, params)
    neuron_params_pn = get_args_from_dict(neuron_modelPN, params)
    neuron_params_ln = get_args_from_dict(neuron_modelLN, params)
    neuron_params_kc = get_args_from_dict(neuron_modelKC, params)

    NG['ORN'] = NeuronGroup(N_glu*ORNperGlu, **neuron_modelORN(**neuron_params_orn), namespace=params, method='euler', name='ORNs')
    NG['ORN'].I0 = I0_PN
    NG['PN'] = NeuronGroup(N_glu, **neuron_modelPN(**neuron_params_pn), namespace=params, method='euler', name='PNs')
    NG['PN'].I0=I0_PN
    NG['LN'] = NeuronGroup(N_glu, **neuron_modelLN(**neuron_params_ln), namespace=params, method='euler', name='LNs')
    NG['LN'].I0=I0_LN
    NG['KC'] = NeuronGroup(N_KC, **neuron_modelKC(**neuron_params_kc), namespace=params, method='euler', name='KCs')
    NG['KC'].I0=I0_KC

    #########################     CONNECTIONS       #########################
    c = dict()

    if input_ng is not None:
        ### input-ORN ###
        c['ORNinputORN'] = Synapses(NG['ORNinput'], NG['ORN'], 'w : siemens', on_pre='g_e+=w', namespace=params)
        for i in np.arange(len(NG['ORN'])):
            #c['ORNinputORN'].connect(i=list(range(i * orn_input_multiplier, (i + 1) * orn_input_multiplier)), j=i)
            c['ORNinputORN'].connect(i=i, j=i)
            c['ORNinputORN'].w = wORNinputORN

    ### ORN-PN ###
    c['ORNPN'] = Synapses(NG['ORN'], NG['PN'], 'w : siemens', on_pre='g_e += w', namespace=params)
    for i in np.arange(N_glu):
        c['ORNPN'].connect(i=list(range(i * ORNperGlu, (i + 1) * ORNperGlu)), j=i)
        c['ORNPN'].w = wORNPN

    ### ORN-LN ###
    c['ORNLN'] = Synapses(NG['ORN'], NG['LN'], 'w : siemens', on_pre='g_e += w', namespace=params)
    for i in np.arange(N_glu):
        c['ORNLN'].connect(i=list(range(i * ORNperGlu, (i + 1) * ORNperGlu)), j=i)
        c['ORNLN'].w = wORNLN

    ### LN-PN ###
    c['LNPN'] = Synapses(NG['LN'], NG['PN'], 'w : siemens', on_pre='g_i -= w', delay=inh_delay, namespace=params)
    c['LNPN'].connect()  # connect_all
    c['LNPN'].w = wLNPN


    ## PN-KC ##
    c['KC'] = Synapses(NG['PN'], NG['KC'], 'w : siemens', on_pre='g_e += w', namespace=params)
    c['KC'].connect(p=PNperKC / float(N_glu))
    c['KC'].w = wPNKC
        # the total number of possible synapses is N_pre*N_post
        # when the connection probability is 0.05 then N_syn = N_pre*N_post*0.05 (on average)
        # every postsynaptic neuron will receive N_syn/N_post synaptic inputs _on average_
        # and every presynaptic input will send out N_syn/N_pre _on average_
        # number of inputs per KC is given by the biominal distribution

    #########################     INITIAL VALUES     #########################
    #NG['PN'].v = np.random.rand(len(NG['PN']))*(V0max-V0min)+V0min
    #NG['LN'].v = np.random.rand(len(NG['LN']))*(V0max-V0min)+V0min
    #NG['KC'].v = np.random.rand(len(NG['KC']))*(V0max-V0min)+V0min

    NG['ORN'].v = np.random.uniform(V0min, V0max, size=len(NG['ORN'])) * volt
    NG['PN'].v = np.random.uniform(V0min, V0max, size=len(NG['PN'])) * volt
    NG['LN'].v = np.random.uniform(V0min, V0max, size=len(NG['LN'])) * volt
    NG['KC'].v = np.random.uniform(V0min, V0max, size=len(NG['KC'])) * volt

    return NG, c